# Generating Random Samples

## Inversion method

If we want to simulate a sample $x$ of the RV $X$, which follows the distribution function $F_X(x)$

1. Calculate the inverse function of $F_X(x)$, $F^{-1}_X(x)$. We may need to use the definition of *generalized inverse* of $F$:

    $F^{-1}(u):=inf\{x:F(x)\geq u\}\, for\,0<u<1$
2. Generate a smaple $u$ from Uniform distribution $U\sim U(0,1)$
3. Set $x:=F^{-1}_X(u)$

### Example

We want to simulate the exponential distribution with $\lambda = 1$ *i.e* $F(x)=1-e^{\lambda x}$, $x>0$
1. the inverse distribution function is $F^{-1}_X(y)=\frac{-ln(1-y)}{\lambda}$
2. $U\sim U(0,1)$
3. $x :=F^{-1}_X(u)=\frac{-ln(1-u)}{\lambda}$

In [12]:
import numpy as np
import matplotlib.pyplot as plt

lamda = 1
N = 100000

x = np.random.exponential(scale = lamda, size = (1,N))

def inv_F(u):
    return -np.log(1-u) / lamda

def main():
    u = np.random.rand(1,N)
    return(inv_F(u))

x_hat = main()

plt.hist(x_hat, bins = 150, density=True)
plt.show()

